# Overview

MONAI bundle usually includes the stored weights of a model, TorchScript model, JSON files that include configs and metadata about the model, information for constructing training, inference, and post-processing transform sequences, plain-text description, legal information, and other data the model creator wishes to include.

For more information about MONAI bundle description: https://docs.monai.io/en/latest/mb_specification.html.

This notebook is step-by-step tutorial to help get started to develop a bundle package. It at least contains a `train.json` config file to construct the training pipeline and may also have a `metadata.json` file to define the metadata information.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/modules/bundles/get_started.ipynb)

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"

In [9]:
import os

from monai.config import print_config
from monai.bundle import ConfigParser

## Setup imports

In [ ]:
# Copyright (c) MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

print_config()

## Setup data directory

Here specify a directory with the `MONAI_DATA_DIRECTORY` environment variable to save downloaded dataset and outputs.

In [6]:
root_dir = os.environ.get("MONAI_DATA_DIRECTORY")
print(f"root dir is: {root_dir}")

root dir is: /workspace/data/medical/


## Download dataset

Downloads and extracts the dataset.  
The dataset comes from http://medicaldecathlon.com/.

In [7]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar"
md5 = "410d4a301da4e5b2f6f86ec3ddba524e"

compressed_file = os.path.join(root_dir, "Task09_Spleen.tar")
data_dir = os.path.join(root_dir, "Task09_Spleen")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

## Define train config - Set imports and input / output environments

Now let's start to define the `train.json` config file for a regular training task.

For more details about the syntax in bundle config, please check: https://github.com/wyli/MONAI/blob/3482-bundle-doc/docs/source/config_syntax.md.

Please note that MONAI bundle doesn't require any hard-code logic in the config, so users can define the config content in any structure.

For the first step, import `os` and `glob` to use in the expressions (start with `$`). Then define input / output environments and enable `cudnn.benchmark` for better performance.

The train and validation image file names are organized into a list of dictionaries.

In [6]:
{
    "imports": [
        "$import glob",
        "$import os"
    ],
    "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
    "cudnn_opt": "$setattr(torch.backends.cudnn, 'benchmark', True)",
    "dataset_dir": "/workspace/data/Task09_Spleen",
    "ckpt_path": "/workspace/data/models/model.pt",
    "datalist": "$list(sorted(glob.glob(@dataset_dir + '/imagesTs/*.nii.gz')))",
    "train_data": "$[{'image': i} for i in @datalist[:-9]]",
    "val_data": "$[{'image': i} for i in @datalist[-9:]]"
}

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)